In [1]:
import csv
import numpy as np
from astropy.table import Table, vstack, hstack, join
from astropy.io import ascii

# Tools

In [2]:
def find_longest(column):
    """Find Longest Value
    Finds the longest value in a data column
    I wouldn't be suprised if there's a built-in function for this, but I like being difficult sometimes :)
    
    Args:
        column (list): column of data with numerical values that can be ~quite~ long
        
    Returns:
        (int): Size of the longest value
    
    """
    longest_value = 0
    for item in column:
        if len(item) > longest_value:
            longest_value = len(item)
            
    return longest_value

In [3]:
def ugly_spacer(longest_value = 0, offset = 0):
    """The Ugly IPAC Spacer
    Generates spaces for IPAC tables, since I think they need to be formatted very specifically to work properly
    I apologize in advance for the sins this function participates in
    
    Args:
        longest_value (int): Length of longest value in a dataset (character-wise)
        offset (int): how many spaces to remove to fit the actual data correctly
        
    Returns:
        (str): String containing the perfect amount of spaces for a beautiful IPAC table
    """
    count = 0
    spacer_list = []
    while count < (longest_value-offset):
        spacer_list.append(' ')
        count += 1
    
    spacer = ''.join(map(str,spacer_list))
    return(spacer)

In [4]:
def min_to_deg(value):
    """Min-to-Deg
    Converts RA/Dec values from deg-min-sec format to decimal degrees, to interface with other data formats
    This function should be valid for any dataset using this format, so here's hoping!
    
    Args:
        value (str): dg-mn-se.conds format coordinates
        
    Returns:
        (float): decimal degree coordinates    
    """
    
    degrees = []
    minutes = []
    seconds = []
    
    coord = [degrees, minutes, seconds]
    index_num = 0
    
    for char in value:
        if char == ' ':
            index_num += 1
        else:
            coord[index_num].append(char)
    
    #I didn't know how to turn this set of lists and ints and such to values
    #until I found this solution on ~the Internet~
    #https://stackoverflow.com/a/490020        
    deg_val = int(''.join(map(str,degrees)))
    min_val = int(''.join(map(str,minutes)))
    sec_val = float(''.join(map(str,seconds)))
    
    decimal_coord = deg_val + min_val/60 + sec_val/60**2
            
    return decimal_coord

In [5]:
def xmm_processor(xmm_data):
    """XMM-Newton Data Processor
    Removes extraneous columns in the data table, as well as converting coordinates to degrees.
    This overwrites the stored table, but does not affect the actual file, so this must be run every time xmm-data is
    read from a file.
    
    Args:
        xmm_data (astropy.Table): XMM-Newton data table that needs processing
        
    Returns:
        n/a    
    """
    #stupid delimiters before columns lead astropy to think there are blank columns at the beginning and end of the data
    #I hate that, but I don't want to alter the data file, so I pulled the following little number to eliminate those:
    #del xmm_data['col0','col21']
    #this thing kills the stupid extraneous columns before 'ra', but I want a sneakier way to do this :(
    #del xmm_data['detid','srcid','name']
    
    count = 0
    for row in xmm_data:
        nu_ra = min_to_deg(row['ra'])
        nu_dec = min_to_deg(row['dec'])
        xmm_data[count]['ra'] = nu_ra
        xmm_data[count]['dec'] = nu_dec
        
        count += 1

# Querying File Generators

In [6]:
def generate_sdss(filename, dataset):
    """SDSS querying table Generator
    Processes a table of XMM-Newton data to produce a workable querying file for the online CrossID database for SDSS.
    I apologize for any sinfulness, this is a first draft of code built to work and not much else
    
    Ags:
        filename (str): desired filename for the output table
        dataset  (astropy.Table): table of XMM-Newton data with coordinates that will be used to construct a query
        
    Returns:
        n/a    
    """

    count = 0
    txt_count = 0
    
    infile = open(filename+str(txt_count)+'.txt', 'w')
    infile.write('name   ra   dec\n')
    
    
    for row in range(len(dataset)):
        if count >= 5000:
            txt_count += 1
            count = 0
            infile.close()
            infile = open(filename+str(txt_count)+'.txt','w')

        ra = float(dataset['ra'][row])
        dec = float(dataset['dec'][row])

        if ra < 0 or dec < 0:
            infile.write(str(row)+'\n')
        else:
            ra_val = str(ra)
            dec_val = str(dec)
            infile.write(str(row)+' '+ra_val+' '+dec_val+'\n')

        count += 1
        
    infile.close()

In [7]:
def generate_ipac(filename, dataset):
    """IPAC Table Generator
    Processes a table of XMM-Newton data to produce a workable querying file for the online Gator database for WISE.
    I apologize for any sinfulness, this is a first draft of code built to work and not much else
    
    Ags:
        filename (str): desired filename for the output IPAC table
        dataset  (astropy.Table): table of XMM-Newton data with coordinates that will be used to construct an IPAC table
        
    Returns:
        n/a    
    """
    #IPAC file for WISE querying, found at the following database:
    #https://irsa.ipac.caltech.edu/cgi-bin/Gator/nph-dd
    #
    #IPAC basic guide here:
    #https://irsa.ipac.caltech.edu/applications/Gator/GatorAid/irsa/upload.html
    infile = open(filename+'.txt', 'w')
    
    ra_longest = find_longest(dataset['ra'])
    dec_longest = find_longest(dataset['dec'])

    #Wanna se a real mess of a table writer? Here you go!
    infile.write('|'+ugly_spacer(ra_longest,2)+'ra|'+ugly_spacer(dec_longest,3)+'dec|\n')
    infile.write('|'+ugly_spacer(ra_longest,6)+'double|'+ugly_spacer(dec_longest,6)+'double|\n')
    infile.write('|'+ugly_spacer(ra_longest)+'|'+ugly_spacer(dec_longest)+'|\n')
    infile.write('|'+ugly_spacer(ra_longest,4)+'null|'+ugly_spacer(dec_longest,4)+'null|\n')
    
    #It gets worse, watch this:
    for row in range(len(dataset)):
        ra_offset = len(dataset['ra'][row])
        dec_offset = len(dataset['dec'][row])
        ra_val = str(dataset['ra'][row])
        dec_val = str(dataset['dec'][row])
        
        infile.write(' '+ugly_spacer(ra_longest,ra_offset)+ra_val+' '+ugly_spacer(dec_longest,dec_offset)+dec_val+' \n')
        
    infile.close()

**Where to get XMM-Newton Data:**

https://heasarc.gsfc.nasa.gov/db-perl/W3Browse/w3table.pl?tablehead=name%3Dxmmssc&Action=More+Options
    
How to grab this data:

    1. Check the "ra","dec", "pn_#_rate (# = 1 to 5)", "m1_#_rate", "m2_#_rate" boxes
    
    2. In the query terms for "ra" and "dec", put ">=0" b/c SDSS hates negative coordinates
    
    3. Go to the very bottom of the page, select your results size in the "Limit Results to:" section
    
    4. Set the output format to be a text table
    
    5. Start the search, and depending on your results size, wait a while. Grab a drink and a snack or smth

**How to successfully query Gator like an absolute madman/madwoman:**

 https://irsa.ipac.caltech.edu/cgi-bin/Gator/nph-dd?catalog=allwise_p3as_psd&mode=html&passproj&
 
    1. Generate an IPAC table using this amazing program
    
    2. Select "Multi-object Search", check the "One to One Match" box, and upload the IPAC table you generated
    
    3. In the "Column Constraints/Output Column Selection" box, deselect all rows except the following:
       "w1mpro", "w2mpro", "w3mpro", "w4mpro" (each of these will have units as "mag", or magnitude)
    
    4. Hit the "Run Query" button at the bottom of the page (or the middle of the page lmao)
    
    5. Let the database work its magic! Get a drink or snack or something if it's taking a while
    
    6. After it finishes, navigate to the results table at the bottom of the page, & click the gear icon("Edit table options")
    
    7. Deselect all but the "ra", "dec", and the rows you selected in step 4
    
    8. Click the floppy disk icon to save the file locally, choose a filename that helps you find the data easily, storing it
       in the same directory as the main program for ease-of-use purposes
    
    9. That's it! You've just queried Gator! Best of luck in your future research endeavours!

**How to grab SDSS data:**

http://cas.sdss.org/dr7/en/tools/crossid/upload.asp

    1. Generate an SDSS query table
    
    2. Upload the file to the link above
    
    3. Input the following SQL query:
    
        SELECT 
           p.ra, p.dec,
           p.modelMag_u, p.modelMag_g, p.modelMag_r, p.modelMag_i, p.modelMag_z 
        FROM #x x, #upload u, PhotoTag p
        WHERE u.up_id = x.up_id and x.objID=p.objID
    
    4. Select CSV-type output
    
    5. Run the query!

# H

In [8]:
xmm_full = ascii.read('BrowseTargets.8264.1539051979', delimiter='|', header_start=2, data_start=3)

In [8]:
xmm = ascii.read('BrowseTargets.28214.1554773283', delimiter='|', header_start=2, data_start=3)

In [9]:
xmm_processor(xmm)

In [10]:
generate_sdss('Apr82019SDSS',xmm)

In [11]:
generate_ipac('Apr82019IPAC',xmm)